# **Refocus**: Host<->Network Append instead of Merge
------------------

1. Host<->Network Append (done)
2. Common Timestamp Column (done)
3. Appy Network TP_Binary based on Source_IP, Dest_IP, Dest_Port (done)
4. Apply Time Window Broad Label - TWB_Binary
5. Apply Time Window Micro Label - TWM_Binary
6. Apply Threat Hunter Expanded Label - THE_Binary

### Import Libraries

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


### Load Network Data

In [7]:
df_zeek = pd.read_json("combined_zeek.log", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
#print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2140, 148)


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conn,bobs.bigwheel.local,zeek,1.588205e+09,CvxbPE3MuO7boUdSc8,10.0.1.6,138.0,10.0.1.255,138.0,udp,NaN,NaN,NaN,NaN,S0,0.0,D,1.0,229.0,0.0,0.0,00:0d:3a:13:e3:69,ff:ff:ff:ff:ff:ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Load Host Data

In [16]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

## 1. Host<->Network Append
------------------

In [23]:
# Replace the below merge with a append
# Join Host<->Network based on 
#df_label = pd.merge(
#    left = df_label,
#    right = df_zeek,
#    how = 'outer',
#    on = ['@timestamp', 'EventID']
#)

In [19]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
#    In [9]: result = pd.concat([df1, df4], axis=1, sort=False)
df_label = df_label.append(df_zeek, ignore_index=True, sort=False)

In [20]:
df_label.shape

(200900, 536)

In [21]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

In [22]:
df_label.tail(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Gr

## 2. Common Timestamp Column
------------------

In [25]:
df_label['time_stamp'] = pd.to_datetime(df_label['@timestamp'])

In [29]:
df_label['time_stamp'].notna().value_counts()

True     198760
False      2140
Name: time_stamp, dtype: int64

In [30]:
df_label['time_stamp'] = df_label["time_stamp"].fillna(pd.to_datetime(df_label['ts']))
    #.apply(lambda x: (pd.to_datetime(x, unit='s', utc=True)))

In [31]:
df_label['time_stamp'].notna().value_counts()

True    200900
Name: time_stamp, dtype: int64

## 3. Appy Network TP_Binary based on Source_IP, Dest_IP, Dest_Port
------------------

### Common Source_IP, Destination_IP, Destination_Port

In [33]:
# Transform Host Data - Similar Data to Single Columns
df_label['Destination_IP'] = df_label['DestinationIp']
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['DestAddress'])
df_label['Source_IP'] = df_label['IpAddress']
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceIp'])
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['SourceAddress'])
df_label['Source_Port'] = df_label['IpPort']
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['SourcePort'])
df_label['Destination_Port'] = df_label['DestinationPort']
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['DestPort'])

In [37]:
# Check Host Equivalent to Net Flows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.groupby(['Source_IP', 'Destination_IP', 'Destination_Port'])['@timestamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


@timestamp  \
Source_IP                      Destination_IP                 Destination_Port               
0:0:0:0:0:0:0:1                0:0:0:0:0:0:0:1                53.0                       2   
                                                              135.0                      4   
                                                              389.0                     74   
                                                              5985.0                     4   
                                                              53681.0                    1   
                                                              53773.0                    1   
                                                              54394.0                    1   
                                                              54783.0                    1   
                                                              55163.0                    1   
                                                              55267.0                    1   
                                                              55852.0                    1   
                                                              56678.0                    1   
                                                              56968.0                    1   
                                                              57566.0                    1   
                                                              58458.0                    1   
                                                              58645.0                    1   
                                                              59207.0                    1   
                                                              60376.0                    1   
                                                              60423.0                    1   
                                                              60731.0                    1   
                                                              60780.0                    1   
                                                              60841.0                    1   
                                                              60964.0                    1   
                                                              62041.0                    1   
                                                              62250.0                    1   
                                                              62280.0                    1   
                                                              63614.0                    1   
                                                              64061.0                    1   
                                                              64489.0                    1   
                                                              64543.0                    1   
                                                              65535.0                    1   
10.0.0.255                     10.0.0.4                       137.0                      1   
                                                              138.0                      1   
10.0.0.4                       10.0.0.255                     137.0                      5   
                                                              138.0                      7   
                               10.0.0.4                       53.0                      48   
                                                              389.0                     41   
                                                              53681.0                    1   
                                                              53858.0                    1   
                                                              54132.0                    1   
                                                              54172.0                    1   
                                      

In [38]:
df_label['Destination_IP'].notna().value_counts()

False    196468
True       4432
Name: Destination_IP, dtype: int64

In [39]:
df_label['Source_IP'] = df_label['Source_IP'].fillna(df_label['id_orig_h'])
df_label['Source_Port'] = df_label['Source_Port'].fillna(df_label['id_orig_p'])
df_label['Destination_IP'] = df_label['Destination_IP'].fillna(df_label['id_resp_h'])
df_label['Destination_Port'] = df_label['Destination_Port'].fillna(df_label['id_resp_p'])

In [40]:
df_label['Destination_IP'].notna().value_counts()

False    195164
True       5736
Name: Destination_IP, dtype: int64

### ID Bad Flows and Tranfer TP_Label and TP_Binary to Network Data

In [47]:
# bad_flows = df.loc[(df['TP_Binary'] == True) & (df[colName].notnull())]['ProcessGuid'].unique()
bad_flows = df_label[(df_label.TP_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())][['EventID', 'TP_Binary', 'TP_Label', 'time_stamp', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']].drop_duplicates()

In [48]:
# Maybe TPE_Binary First? or Just TP_Binary bad flows, THEN TPE_Binary
bad_flows

,EventID,TP_Binary,TP_Label,time_stamp,Source_IP,Source_Port,Destination_IP,Destination_Port
12637,5156.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02 03:09:29.935000+00:00,10.0.1.4,59961,10.0.1.6,5985.0
14743,3.0,True,"0, 8.A.1. Remote System Discovery",2020-05-02 03:09:07.649000+00:00,10.0.1.4,59955,10.0.0.4,389.0
16280,5156.0,True,"0, 3.B.3. Commonly Used Port",2020-05-02 02:58:47.195000+00:00,10.0.1.4,59845,192.168.0.5,443.0
24641,3.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02 03:09:31.065000+00:00,10.0.1.4,59960,10.0.1.6,5985.0
29064,5156.0,True,"0, 8.A.1. Remote System Discovery",2020-05-02 03:09:05.633000+00:00,10.0.1.4,59955,10.0.0.4,389.0
29065,5156.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02 03:09:32.125000+00:00,10.0.1.4,59962,10.0.1.6,5985.0
49253,3.0,True,"0, 8.A.2. Remote System Discovery",2020-05-02 03:09:33.128000+00:00,10.0.1.4,59962,10.0.1.6,5985.0
53904,5156.0,True,"0, 8.A.1. Remote System Discovery",2020-05-02 03:09:05.635000+00:00,10.0.1.4,59957,10.0.0.4,389.0
62937,3.0,True,"0, 3.B.3. Commonly Used Port",2020-05-02 02:58:48.298000+00:00,10.0.1.4,59845,192.168.0.5,443.0
74758,3.0,True,"0, 8.A.1. Remote System Discovery",2020-05-02 03:09:07.650000+00:00,10.0.1.4,59957,10.0.0.4,389.0


In [59]:
# Check Host Equivalent to Net Flows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(bad_flows.groupby(['TP_Label', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,,,,time_stamp,EventID,TP_Binary,Source_IP,Destination_IP,Destination_Port
TP_Label,Source_IP,Destination_IP,Destination_Port,,,,,,
"0, 1.A.3. Uncommonly Used Port",10.0.1.4,192.168.0.5,1234.0,2,2,2,2,2,2
"0, 3.B.3. Commonly Used Port",10.0.1.4,192.168.0.5,443.0,4,4,4,4,4,4
"0, 8.A.1. Remote System Discovery",10.0.1.4,10.0.0.4,389.0,4,4,4,4,4,4
"0, 8.A.2. Remote System Discovery",10.0.1.4,10.0.1.6,5985.0,8,8,8,8,8,8


In [53]:
# Check zeek netflows
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[df_label['id_orig_h'].notna()].groupby(['Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


time_stamp  EventID  TP_Binary  \
Source_IP Destination_IP Destination_Port                                   
10.0.1.4  10.0.0.4       53.0                      52        0          0   
                         88.0                      32        0          0   
                         123.0                      3        0          0   
                         135.0                      6        0          0   
                         389.0                     12        0          0   
                         445.0                     76        0          0   
                         49667.0                   10        0          0   
          10.0.1.255     137.0                      4        0          0   
                         138.0                      4        0          0   
          10.0.1.6       80.0                       6        0          0   
                         135.0                     20        0          0   
                         445.0                     42        0          0   
                         5985.0                     8        0          0   
                         55121.0                   36        0          0   
          192.168.0.4    80.0                      68        0          0   
                         139.0                      5        0          0   
                         445.0                      5        0          0   
          192.168.0.5    443.0                      5        0          0   
                         1234.0                     1        0          0   
10.0.1.6  10.0.0.4       53.0                      32        0          0   
                         88.0                      18        0          0   
                         123.0                      3        0          0   
                         135.0                      6        0          0   
                         389.0                     18        0          0   
                         445.0                     60        0          0   
                         49667.0                   11        0          0   
          10.0.1.255     137.0                      4        0          0   
                         138.0                      4        0          0   
          192.168.0.4    8443.0                   753        0          0   

                                           Source_IP  Destination_IP  \
Source_IP Destination_IP Destination_Port                              
10.0.1.4  10.0.0.4       53.0                     52              52   
                         88.0                     32              32   
                         123.0                     3               3   
                         135.0                     6               6   
                         389.0                    12              12   
                         445.0                    76              76   
                         49667.0                  10              10   
          10.0.1.255     137.0                     4               4   
                         138.0                     4               4   
          10.0.1.6       80.0                      6               6   
                         135.0                    20              20   
                         445.0                    42              42   
                         5985.0                    8               8   
                         55121.0                  36              36   
          192.168.0.4    80.0                     68              68   
                         139.0                     5               5   
                         445.0                     5               5   
          192.168.0.5    443.0                     5               5   
                         1234.0                    1               1   
10.0.1.6  10.0.0.4       53.0                     32              32   
                         88.0                     18              18   

In [60]:
#bad_flows = df_label[(df_label.TP_Binary==True) & (df_label.Source_IP.notnull()) & (df_label.Destination_IP.notnull())][['EventID', 'TP_Binary', 'TP_Label', 'time_stamp', 'Source_IP', 'Source_Port', 'Destination_IP', 'Destination_Port']].drop_duplicates()
df_label['TP_Label'].loc[(df_label.id_orig_h == '10.0.1.4') & (df_label.id_resp_h == '192.168.0.5') & (df_label.id_resp_p == 1234.0)] = '0, 1.A.3. Uncommonly Used Port'

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [61]:
df_label['TP_Label'].loc[(df_label.id_orig_h == '10.0.1.4') & (df_label.id_resp_h == '192.168.0.5') & (df_label.id_resp_p == 1234.0)]

200502    0, 1.A.3. Uncommonly Used Port
Name: TP_Label, dtype: object

In [64]:
df_label['TP_Label'].loc[(df_label.id_orig_h == '10.0.1.4') & (df_label.id_resp_h == '192.168.0.5') & (df_label.id_resp_p == 443.0)] = '0, 3.B.3. Commonly Used Port'

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [67]:
df_label['TP_Label'].loc[(df_label.id_orig_h == '10.0.1.4') & (df_label.id_resp_h == '10.0.0.4') & (df_label.id_resp_p == 389.0)] = '0, 8.A.1. Remote System Discovery'

In [69]:
df_label['TP_Label'].loc[(df_label.id_orig_h == '10.0.1.4') & (df_label.id_resp_h == '10.0.1.6') & (df_label.id_resp_p == 5985.0)] = '0, 8.A.2. Remote System Discovery'

In [71]:
df_label['TP_Label'] = df_label.TP_Label.fillna(0)

In [72]:
df_label['TP_Binary'] = df_label.TP_Label!='0'

In [74]:
df_label['TP_Binary'].loc[df_label.EventID.notna()].value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [75]:
df_label['TP_Binary'].loc[df_label['@system'].notna()].value_counts()

True    2140
Name: TP_Binary, dtype: int64

In [76]:
df_label['TP_Binary'] = df_label.TP_Binary.fillna(False)

In [77]:
df_label['TP_Binary'].loc[df_label['@system'].notna()].value_counts()

True    2140
Name: TP_Binary, dtype: int64

In [78]:
# Re-Check zeek netflows with TP_Binary, TP_Label
#     -> TP_Binary is not correct. TP_Label seems right.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[df_label['id_orig_h'].notna()].groupby(['TP_Binary', 'TP_Label', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


time_stamp  \
TP_Binary TP_Label                          Source_IP Destination_IP Destination_Port               
True      0                                 10.0.1.4  10.0.0.4       53.0                      52   
                                                                     88.0                      32   
                                                                     123.0                      3   
                                                                     135.0                      6   
                                                                     445.0                     76   
                                                                     49667.0                   10   
                                                      10.0.1.255     137.0                      4   
                                                                     138.0                      4   
                                                      10.0.1.6       80.0                       6   
                                                                     135.0                     20   
                                                                     445.0                     42   
                                                                     55121.0                   36   
                                                      192.168.0.4    80.0                      68   
                                                                     139.0                      5   
                                                                     445.0                      5   
                                            10.0.1.6  10.0.0.4       53.0                      32   
                                                                     88.0                      18   
                                                                     123.0                      3   
                                                                     135.0                      6   
                                                                     389.0                     18   
                                                                     445.0                     60   
                                                                     49667.0                   11   
                                                      10.0.1.255     137.0                      4   
                                                                     138.0                      4   
                                                      192.168.0.4    8443.0                   753   
          0, 1.A.3. Uncommonly Used Port    10.0.1.4  192.168.0.5    1234.0                     1   
          0, 3.B.3. Commonly Used Port      10.0.1.4  192.168.0.5    443.0                      5   
          0, 8.A.1. Remote System Discovery 10.0.1.4  10.0.0.4       389.0                     12   
          0, 8.A.2. Remote System Discovery 10.0.1.4  10.0.1.6       5985.0                     8   

                                                                                       EventID  \
TP_Binary TP_Label                          Source_IP Destination_IP Destination_Port            
True      0                                 10.0.1.4  10.0.0.4       53.0                    0   
                                                                     88.0                    0   
                                                                     123.0                   0   
                                                                     135.0                   0   
                                                                     445.0                   0   
                                                                     49667.0                 0   
                                                      10.0.1.255     137.0                   0   
                                                                     138.

In [79]:
df_label['TP_Binary'] = df_label.TP_Label!=0

In [80]:
# Re-Check zeek netflows with TP_Binary, TP_Label
#     -> Fixed
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[df_label['id_orig_h'].notna()].groupby(['TP_Binary', 'TP_Label', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


time_stamp  \
TP_Binary TP_Label                          Source_IP Destination_IP Destination_Port               
False     0                                 10.0.1.4  10.0.0.4       53.0                      52   
                                                                     88.0                      32   
                                                                     123.0                      3   
                                                                     135.0                      6   
                                                                     445.0                     76   
                                                                     49667.0                   10   
                                                      10.0.1.255     137.0                      4   
                                                                     138.0                      4   
                                                      10.0.1.6       80.0                       6   
                                                                     135.0                     20   
                                                                     445.0                     42   
                                                                     55121.0                   36   
                                                      192.168.0.4    80.0                      68   
                                                                     139.0                      5   
                                                                     445.0                      5   
                                            10.0.1.6  10.0.0.4       53.0                      32   
                                                                     88.0                      18   
                                                                     123.0                      3   
                                                                     135.0                      6   
                                                                     389.0                     18   
                                                                     445.0                     60   
                                                                     49667.0                   11   
                                                      10.0.1.255     137.0                      4   
                                                                     138.0                      4   
                                                      192.168.0.4    8443.0                   753   
True      0, 1.A.3. Uncommonly Used Port    10.0.1.4  192.168.0.5    1234.0                     1   
          0, 3.B.3. Commonly Used Port      10.0.1.4  192.168.0.5    443.0                      5   
          0, 8.A.1. Remote System Discovery 10.0.1.4  10.0.0.4       389.0                     12   
          0, 8.A.2. Remote System Discovery 10.0.1.4  10.0.1.6       5985.0                     8   

                                                                                       EventID  \
TP_Binary TP_Label                          Source_IP Destination_IP Destination_Port            
False     0                                 10.0.1.4  10.0.0.4       53.0                    0   
                                                                     88.0                    0   
                                                                     123.0                   0   
                                                                     135.0                   0   
                                                                     445.0                   0   
                                                                     49667.0                 0   
                                                      10.0.1.255     137.0                   0   
                                                                     138.

In [81]:
# Check Host and Network TP_Binary distribution
df_label['TP_Binary'].loc[df_label.EventID.notna()].value_counts()

True    198760
Name: TP_Binary, dtype: int64

In [85]:
df_label.dtypes

@timestamp           object
@version            float64
Category             object
Channel              object
EventID             float64
                     ...   
time_stamp           object
Destination_IP       object
Source_IP            object
Source_Port          object
Destination_Port    float64
Length: 541, dtype: object

In [84]:
df_label['TP_Label'].dtypes

dtype('O')

In [87]:
df_label = df_label.astype({'TP_Label': 'object'})

In [88]:
df_label['TP_Label'].dtypes

dtype('O')

In [89]:
# Check Host and Network TP_Binary distribution
#     -> only network TP_Binary correct
df_label['TP_Binary'].value_counts()

True     198786
False      2114
Name: TP_Binary, dtype: int64

In [90]:
df_label['TP_Binary'].loc[df_label.EventID.notna()] = df_label.TP_Label!='0'

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [91]:
df_label['TP_Binary'].loc[df_label['id_orig_h'].notna()] = df_label.TP_Label!=0

In [92]:
# Check Host and Network TP_Binary distribution
#     -> Suspected issue with TP_Label data type. Likely the pandas dataframe inference of schema: int vs string vs object
#     -> Seems fixed.
df_label['TP_Binary'].value_counts()

False    197436
True       3464
Name: TP_Binary, dtype: int64

In [93]:
# Re-Check zeek netflows with TP_Binary, TP_Label
#     -> Fixed
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.groupby(['TP_Binary', 'TP_Label'])['time_stamp', 'EventID', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


time_stamp  \
TP_Binary TP_Label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
False     0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           2114   
          0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         195322   
True      0, 1.A.1. User Execution                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       1   
          0, 1.A.1. User Execution, 1.A.2. Masquerading                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  2   
          0, 1.A.3. Uncommonly Used Port                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 3   
          0, 1.A.4. Standard Cryptographic Protocol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      1   
          0, 1.B.1. Command-Line Interface                                                                                                                                                                                                                                                                      

In [94]:
# Fixing Data Type for TP_Label
df_label['TP_Label'].replace(0, '0', inplace=True)

In [95]:
df_label['TP_Binary'] = df_label.TP_Label!='0'

In [97]:
# Re-Check zeek netflows with TP_Binary, TP_Label
#     -> Fixed. Notice the difference between EventID count and id_orig_h count. 
#     -> Shows Host and zeek TP_Label->TP_Binary are both making the correct transformation.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.groupby(['TP_Binary', 'TP_Label'])['EventID', 'id_orig_h', 'TP_Binary', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


EventID  \
TP_Binary TP_Label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
False     0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      195322   
True      0, 1.A.1. User Execution                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    1   
          0, 1.A.1. User Execution, 1.A.2. Masquerading                                                                                                                                                                                                                                                                                                                                                                                                                                                                               2   
          0, 1.A.3. Uncommonly Used Port                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              2   
          0, 1.A.4. Standard Cryptographic Protocol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1   
          0, 1.B.1. Command-Line Interface                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            3   
          0, 1.B.1. Command-Line Interface, 2.A.1. File and Directory Discovery                                                                                                                                                                                                                                                         

## 4. Apply Time Window Broad Label - TWB_Binary
------------------

In [99]:
# Looks to be what we want. start and end type per label per system.
df_label.sort_values('time_stamp').groupby(['TP_Binary', 'Hostname'])['time_stamp'].agg(['first','last', 'count'])

TypeError: '<' not supported between instances of 'int' and 'Timestamp'

In [100]:
# More obstacles with format translation between columns after append. Still suspect pandas schema inference.
#     -> 2. Apply Common Timestamp Column Failed.
#df_zeek['ts_pd'] = df_zeek["ts"].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True)))
#df_label['ts_pd'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x, unit='s')))

In [103]:
df_label['time_stamp'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x)))

In [109]:
df_label['ts'].apply(lambda x: (pd.Timestamp(x)))

0                                  NaT
1                                  NaT
2                                  NaT
3                                  NaT
4                                  NaT
                      ...             
200895   1970-01-01 00:00:01.588206741
200896   1970-01-01 00:00:01.588206794
200897   1970-01-01 00:00:01.588206904
200898   1970-01-01 00:00:01.588205718
200899   1970-01-01 00:00:01.588205718
Name: ts, Length: 200900, dtype: datetime64[ns]

In [117]:
df_label['time_stamp'] = df_label['time_stamp'].fillna(df_label['ts'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True))))

In [118]:
df_label['time_stamp']

0           2020-05-02 03:20:47.897000+00:00
1           2020-05-02 03:19:36.692000+00:00
2           2020-05-02 03:19:37.733000+00:00
3           2020-05-02 03:19:46.143000+00:00
4           2020-05-02 03:19:46.157000+00:00
                         ...                
200895   2020-04-30 00:32:21.989507914+00:00
200896   2020-04-30 00:33:14.863337994+00:00
200897   2020-04-30 00:35:04.274806023+00:00
200898   2020-04-30 00:15:18.382302046+00:00
200899   2020-04-30 00:15:18.757731915+00:00
Name: time_stamp, Length: 200900, dtype: datetime64[ns, UTC]

### Common Timestamp Column Fixed.

### Apply Time Window Broad to Host Events

In [130]:
# Only Showing Host data for Hostname hits.
#    -> Need Common Hostname or System or Source_IP or something
df_label.sort_values('time_stamp').groupby(['TP_Binary', 'Hostname'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary Hostname                                                  
False     NASHUA.dmevals.local   2020-05-02 02:55:26.494000+00:00   
          NEWYORK.dmevals.local  2020-05-02 02:55:27.499000+00:00   
          SCRANTON.dmevals.local 2020-05-02 02:55:28.511000+00:00   
          UTICA.dmevals.local    2020-05-02 02:55:26.493000+00:00   
True      NASHUA.dmevals.local   2020-05-02 03:09:29.929000+00:00   
          NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00   
          SCRANTON.dmevals.local 2020-05-02 02:55:57.730000+00:00   

                                                             last   count  
TP_Binary Hostname                                                         
False     NASHUA.dmevals.local   2020-05-02 03:28:20.170000+00:00   29014  
          NEWYORK.dmevals.local  2020-05-02 03:28:15.147000+00:00   23934  
          SCRANTON.dmevals.local 2020-05-02 03:28:13.119000+00:00  130403  
          UTICA.dmevals.local    2020-05-02 03:28:20.170000+00:00   11971  
True      NASHUA.dmevals.local   2020-05-02 03:17:53.817000+00:00      44  
          NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00       1  
          SCRANTON.dmevals.local 2020-05-02 03:19:53.189000+00:00    3393

In [127]:
# Check TP_Binary label for events within the 'malicious' time window
df_label.loc[(df_label['time_stamp'] >= '2020-05-02 03:09:29.929000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:17:53.817000+00:00') &
             (df_label['Hostname'] == 'NASHUA.dmevals.local')
            ]['TP_Binary'].value_counts()

False    11014
True        44
Name: TP_Binary, dtype: int64

In [128]:
# Create new column for broad time window labelling - start<>end time and hostname
df_label['TWB_Binary'] = False
# Label malicious: start<>end time and hostname
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 03:09:29.929000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:17:53.817000+00:00') &
             (df_label['Hostname'] == 'NASHUA.dmevals.local')
            ] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [129]:
df_label['TWB_Binary'].value_counts()

False    189842
True      11058
Name: TWB_Binary, dtype: int64

In [131]:
# Implement Label for other hostname-time windows
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 03:04:05.749000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:04:05.749000+00:00') &
             (df_label['Hostname'] == 'NEWYORK.dmevals.local')
            ] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [132]:
df_label['TWB_Binary'].value_counts()

False    189837
True      11063
Name: TWB_Binary, dtype: int64

In [133]:
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 02:55:57.730000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:19:53.189000+00:00') &
             (df_label['Hostname'] == 'SCRANTON.dmevals.local')
            ] = True

In [134]:
df_label['TWB_Binary'].value_counts()

False    111336
True      89564
Name: TWB_Binary, dtype: int64

### Apply Time Window Broad to Network Events

In [136]:
df_label.loc[(df_label['id_orig_h'].notna())].sort_values('time_stamp').groupby(['TP_Binary', 'Source_IP', 'Destination_IP'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary Source_IP Destination_IP                                       
False     10.0.1.4  10.0.0.4       2020-04-30 00:06:38.899709940+00:00   
                    10.0.1.255     2020-04-30 00:09:01.189415932+00:00   
                    10.0.1.6       2020-04-30 00:35:25.606390953+00:00   
                    192.168.0.4    2020-04-30 00:32:14.891381979+00:00   
          10.0.1.6  10.0.0.4       2020-04-30 00:06:39.824369907+00:00   
                    10.0.1.255     2020-04-30 00:08:43.385543108+00:00   
                    192.168.0.4    2020-04-30 00:40:44.730953932+00:00   
True      10.0.1.4  10.0.0.4       2020-04-30 00:12:00.847908020+00:00   
                    10.0.1.6       2020-04-30 00:33:26.283442020+00:00   
                    192.168.0.5    2020-04-30 00:08:02.361653090+00:00   

                                                                  last  count  
TP_Binary Source_IP Destination_IP                                             
False     10.0.1.4  10.0.0.4       2020-04-30 00:43:58.389343023+00:00    179  
                    10.0.1.255     2020-04-30 00:45:00.760719061+00:00      8  
                    10.0.1.6       2020-04-30 00:45:00.056051970+00:00    104  
                    192.168.0.4    2020-04-30 00:35:25.658109903+00:00     78  
          10.0.1.6  10.0.0.4       2020-04-30 00:43:59.015400887+00:00    148  
                    10.0.1.255     2020-04-30 00:44:42.662991047+00:00      8  
                    192.168.0.4    2020-04-30 00:44:44.752271891+00:00    753  
True      10.0.1.4  10.0.0.4       2020-04-30 00:35:04.410170078+00:00     12  
                    10.0.1.6       2020-04-30 00:33:29.289120913+00:00      8  
                    192.168.0.5    2020-04-30 00:15:18.751499891+00:00      6

In [138]:
# Check TP_Binary label for events within the 'malicious' time window
df_label.loc[(df_label['time_stamp'] >= '2020-04-30 00:12:00.847908020+00:00') & 
             (df_label['time_stamp'] <= '2020-04-30 00:35:04.410170078+00:00') &
             ((df_label['Source_IP'] == '10.0.1.4') & (df_label['Destination_IP'] == '10.0.0.4') |
             (df_label['Source_IP'] == '10.0.0.4') & (df_label['Destination_IP'] == '10.0.1.4'))
            ]['TP_Binary'].value_counts()

False    138
True      12
Name: TP_Binary, dtype: int64

In [139]:
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-04-30 00:12:00.847908020+00:00') & 
                           (df_label['time_stamp'] <= '2020-04-30 00:35:04.410170078+00:00') &
                           ((df_label['Source_IP'] == '10.0.1.4') & (df_label['Destination_IP'] == '10.0.0.4') |
                            (df_label['Source_IP'] == '10.0.0.4') & (df_label['Destination_IP'] == '10.0.1.4'))
                          ] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [140]:
df_label['TWB_Binary'].value_counts()

False    111186
True      89714
Name: TWB_Binary, dtype: int64

In [143]:
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-04-30 00:33:26.283442020+00:00') & 
                           (df_label['time_stamp'] <= '2020-04-30 00:33:29.289120913+00:00') &
                           ((df_label['Source_IP'] == '10.0.1.4') & (df_label['Destination_IP'] == '10.0.1.6') |
                            (df_label['Source_IP'] == '10.0.1.6') & (df_label['Destination_IP'] == '10.0.1.4'))
                          ] = True

In [144]:
df_label['TWB_Binary'].value_counts()

False    111178
True      89722
Name: TWB_Binary, dtype: int64

In [145]:
df_label['TWB_Binary'].loc[(df_label['time_stamp'] >= '2020-04-30 00:08:02.361653090+00:00') & 
                           (df_label['time_stamp'] <= '2020-04-30 00:15:18.751499891+00:00') &
                           ((df_label['Source_IP'] == '10.0.1.4') & (df_label['Destination_IP'] == '192.168.0.5') |
                            (df_label['Source_IP'] == '192.168.0.5') & (df_label['Destination_IP'] == '10.0.1.4'))
                          ] = True

In [146]:
df_label['TWB_Binary'].value_counts()

False    111172
True      89728
Name: TWB_Binary, dtype: int64

## 5. Apply Time Window Micro Label - TWM_Binary
------------------

In [155]:
# Maybe start with a unified 'hostname' identifier field.
#    -> Hostname or 'Source_IP + Destination_IP'
#    -> zeek uid might be better => "As a connection is processed by Zeek, a unique identifier is assigned to each session"
df_label['uid'].value_counts()

CIVc4v3hpPmaYo6fld    44
CzUc9syGEsRry0zWl     44
CPKuRr3fL4F8tgqXQ1    41
CrAK8T22lnPVqRKChd    14
C1OVTv1ML8aw5ffxn6    13
                      ..
CdJJLM1fznopMYmPu3     1
CjVkjaacImgMSLB8d      1
CuRbE21APSQo2qd6rk     1
CbIjvF1imuPAgzeN55     1
Ct1ULKeuPYKyOYP7b      1
Name: uid, Length: 613, dtype: int64

In [161]:
df_label.loc[df_label['TP_Binary'] == True].sort_values('time_stamp').groupby(['TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary uid                Source_IP Destination_IP Destination_Port                                       
True      C0XYv11LLkSOaKmcOa 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:29.274476051+00:00   
          C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0            2020-04-30 00:33:14.840056896+00:00   
          C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:01.204940081+00:00   
          C69FpH1T94ougWFTdb 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:29.289120913+00:00   
          C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:24:18.967320919+00:00   
          C9dhLYJAxTxamgyc6  10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.335884094+00:00   
          CAJolD1twy3OPmoKa3 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:28.228626013+00:00   
          CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:01.319463015+00:00   
          CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:26.297113895+00:00   
          CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0            2020-04-30 00:15:18.365298986+00:00   
          CQ4h1Q3AZQHI2vFQZa 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.343384981+00:00   
          CRTaeg2G66cIB9TRm6 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:28.214093924+00:00   
          CRxmG32y1yTI2Odtt9 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:01.170243025+00:00   
          CUFN2104UUWgAE48   10.0.1.4  192.168.0.5    1234.0           2020-04-30 00:08:02.361653090+00:00   
          CWHEjpiltZznVrMaa  10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:27.786571026+00:00   
          CgFqEH2AArPQ4DLOYf 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:26.283442020+00:00   
          CjqEFp2wB12h7DEGt2 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:27.772180080+00:00   
          Cmkszo2NmFqZrTjZ9k 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.332489014+00:00   
          CnfxOnz9EEuMywvih  10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:00.847908020+00:00   
          Cq9APi3Qa52XoUAXF7 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.164742947+00:00   
          CzJQsD4vDzP1O4LDK1 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:33:14.812469006+00:00   
          CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0            2020-04-30 00:15:18.749670982+00:00   
          Czitkj3FissLiylzKe 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.410170078+00:00   

                                                                                                      last  \
TP_Binary uid                Source_IP Destination_IP Destination_Port                                       
True      C0XYv11LLkSOaKmcOa 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:29.274476051+00:00   
          C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0            2020-04-30 00:33:14.840056896+00:00   
          C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:01.204940081+00:00   
          C69FpH1T94ougWFTdb 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:29.289120913+00:00   
          C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:24:18.967320919+00:00   
          C9dhLYJAxTxamgyc6  10.0.1.4  10.0.0.4       389.0            2020-04-30 00:35:04.335884094+00:00   
          CAJolD1twy3OPmoKa3 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:28.228626013+00:00   
          CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0            2020-04-30 00:12:01.319463015+00:00   
          CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0           2020-04-30 00:33:26.297113895+00:00   
          CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0            2020-04-30 00:15:18.392851114+00:00   
          CQ4h1Q3AZQHI2vFQZa 1

In [162]:
df_label['Hostname'].value_counts()

SCRANTON.dmevals.local    133796
NASHUA.dmevals.local       29058
NEWYORK.dmevals.local      23935
UTICA.dmevals.local        11971
Name: Hostname, dtype: int64

In [163]:
df_label['Hostname'] = df_label['Hostname'].fillna(df_label['uid'])

In [164]:
df_label['Hostname'].value_counts()

SCRANTON.dmevals.local    133796
NASHUA.dmevals.local       29058
NEWYORK.dmevals.local      23935
UTICA.dmevals.local        11971
CzUc9syGEsRry0zWl             44
                           ...  
CIc3d33Zzv9Bv7y24f             1
CHTdh64AsyEGqj0Ca              1
CMt7gy4720J7agfEJb             1
CAfhNb15CEQDIDaGo5             1
Ct1ULKeuPYKyOYP7b              1
Name: Hostname, Length: 617, dtype: int64

In [165]:
df_label['time_stamp']

0           2020-05-02 03:20:47.897000+00:00
1           2020-05-02 03:19:36.692000+00:00
2           2020-05-02 03:19:37.733000+00:00
3           2020-05-02 03:19:46.143000+00:00
4           2020-05-02 03:19:46.157000+00:00
                         ...                
200895   2020-04-30 00:32:21.989507914+00:00
200896   2020-04-30 00:33:14.863337994+00:00
200897   2020-04-30 00:35:04.274806023+00:00
200898   2020-04-30 00:15:18.382302046+00:00
200899   2020-04-30 00:15:18.757731915+00:00
Name: time_stamp, Length: 200900, dtype: datetime64[ns, UTC]

In [174]:
# ctime format naturally cuts data at the 'second' unit level. 2030 unique seconds in the dataset
df_label['time_stamp'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True).ctime())).value_counts()

Sat May  2 03:20:37 2020    2000
Sat May  2 03:00:37 2020    1919
Sat May  2 03:20:38 2020    1500
Sat May  2 03:20:17 2020    1500
Sat May  2 03:20:46 2020    1500
                            ... 
Thu Apr 30 00:35:03 2020       1
Thu Apr 30 00:33:44 2020       1
Thu Apr 30 00:08:02 2020       1
Thu Apr 30 00:19:21 2020       1
Sat May  2 03:00:17 2020       1
Name: time_stamp, Length: 2030, dtype: int64

In [176]:
df_label['time_stamp_seconds'] = df_label['time_stamp'].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True).ctime()))

In [177]:
bad_seconds = df_label[df_label.TP_Binary==True][['time_stamp_seconds', 'Hostname']].drop_duplicates()

In [179]:
bad_seconds['TWM_Binary'] = True

In [180]:
bad_seconds

,time_stamp_seconds,Hostname,TWM_Binary
42,Sat May 2 03:03:16 2020,SCRANTON.dmevals.local,True
184,Sat May 2 03:00:37 2020,SCRANTON.dmevals.local,True
185,Sat May 2 03:00:28 2020,SCRANTON.dmevals.local,True
194,Sat May 2 03:00:26 2020,SCRANTON.dmevals.local,True
203,Sat May 2 03:00:30 2020,SCRANTON.dmevals.local,True
...,...,...,...
200564,Thu Apr 30 00:35:04 2020,Cq9APi3Qa52XoUAXF7,True
200565,Thu Apr 30 00:35:04 2020,Czitkj3FissLiylzKe,True
200576,Thu Apr 30 00:35:04 2020,CQ4h1Q3AZQHI2vFQZa,True
200586,Thu Apr 30 00:33:14 2020,C0vu7WGlpKLjL8Qhc,True


In [181]:
df_label.shape

(200900, 543)

In [182]:
df_label = pd.merge(
    left = df_label,
    right = bad_seconds,
    how = 'outer',
    on = ['time_stamp_seconds', 'Hostname']
)

In [183]:
df_label.shape

(200900, 544)

In [185]:
df_label['TWM_Binary'].value_counts()

True    19404
Name: TWM_Binary, dtype: int64

In [186]:
df_label.TWM_Binary = df_label.TWM_Binary.fillna(False)

In [187]:
df_label['TWM_Binary'].value_counts()

False    181496
True      19404
Name: TWM_Binary, dtype: int64

## 6. Apply Threat Hunter Expanded Label - THE_Binary
------------------
- There is a problem with the TP_Binary label. 
- My implementation of the Threat Hunting approach is so specific that a 'malicious' ProcessGuid could me marked True in a given record but the same ProcessGuid is found in plenty of other records marked False.
- The Threat Hunting approach (in the most abstract) is take alert as entry point into dataset, pivot to discover new 'malicious' selectors, pivot using the new selectors, rinse, repeat (not unlike semi-supervised learning cough cough).
- To fully demonstrate the Threat Hunting approach it is important to exapand the label based on the derived 'malicious' selectors.
- This label will be based on the list of 'malicious' selectors (objects/nodes/etc) - like, processes, files, users, etc.
- This label will be better suited for classical ML scoring (accuracy, precision, f1-score, etc).

In [194]:
# We can use ProcessGuid as a unique label based on a GUID's property of being unique within an Active Directory Domain.
bad_processes = df_label[(df_label.TP_Binary==True) & (df_label.ProcessGuid.notna())]['ProcessGuid'].unique()

In [196]:
# Merge method for TWM_Binary took forever. Looking for new method.
bad_processes

array(['{47ab858c-e6b2-5eac-4d00-000000000500}',
       '{47ab858c-e2f2-5eac-d203-000000000400}',
       '{47ab858c-e23d-5eac-c603-000000000400}',
       '{47ab858c-e442-5eac-ed03-000000000400}',
       '{47ab858c-e442-5eac-ec03-000000000400}',
       '{47ab858c-e374-5eac-d803-000000000400}',
       '{47ab858c-e13c-5eac-a903-000000000400}',
       '{47ab858c-e144-5eac-ab03-000000000400}',
       '{47ab858c-e1e4-5eac-b803-000000000400}',
       '{5aa8ec29-e56d-5eac-6e03-000000000400}',
       '{5aa8ec29-e624-5eac-7f03-000000000400}',
       '{5aa8ec29-e5b7-5eac-7703-000000000400}',
       '{47ab858c-e49e-5eac-ef03-000000000400}',
       '{47ab858c-e49f-5eac-f003-000000000400}',
       '{47ab858c-e1f8-5eac-bc03-000000000400}',
       '{5aa8ec29-e5e4-5eac-7a03-000000000400}',
       '{5aa8ec29-e5b8-5eac-7903-000000000400}',
       '{5aa8ec29-e4ec-5eac-6803-000000000400}',
       '{47ab858c-e1e3-5eac-b603-000000000400}',
       '{47ab858c-e49f-5eac-f103-000000000400}',
       '{5aa8ec29-ca

In [201]:
df_label['THE_Binary'] = df_label['TP_Binary']
df_label['THE_Binary'].loc[(df_label.THE_Binary == False) & (df_label.ProcessGuid.isin(bad_processes))] = True

In [252]:
df_label['TP_Binary'].value_counts()

False    197436
True       3464
Name: TP_Binary, dtype: int64

In [202]:
df_label['THE_Binary'].value_counts()

False    174628
True      26272
Name: THE_Binary, dtype: int64

In [242]:
# Files is a little tougher. There isn't a GUID for files...
# Plus Image rolls up a bunch of benign binaries used for malicious purposes on a given host.
df_label.loc[(df_label.TP_Binary == True)]['Image'].value_counts()

C:\Program Files\SysinternalsSuite\sdelete64.exe             18
C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe    17
C:\windows\system32\services.exe                              7
C:\Windows\Temp\python.exe                                    6
C:\Windows\System32\rundll32.exe                              5
C:\windows\System32\WindowsPowerShell\v1.0\powershell.exe     4
C:\ProgramData\victim\â€®cod.3aka3.scr                        4
C:\windows\system32\WindowsPowerShell\v1.0\powershell.exe     3
C:\Windows\Temp\sdelete64.exe                                 3
C:\Windows\System32\cmd.exe                                   2
C:\Windows\Temp\Rar.exe                                       1
C:\Windows\System32\javamtsup.exe                             1
C:\windows\system32\cmd.exe                                   1
C:\windows\system32\svchost.exe                               1
System                                                        1
C:\Program Files\SysinternalsSuite\acces

In [221]:
# TargetFilename we can isolate only those files:
#    Event ID 2: A process changed a file creation time
#    Event ID 11: FileCreate
#    Event ID 15: FileCreateStreamHash
#    Event ID 23: FileDelete (A file delete was detected)
# that carry a TP_Binary == True label. 
# All these events are sysmon generated. Comperable organic Windows events are (Open 4656, Access 4663, Close 4658)
# Notice the ZZZZ.ZZZ files. 
#    -> These are evidence of the defense evasion technique to throw characters into event creation that it can't handle.
#    -> But we can use this fact as a clear marker of 'malicious' behavior.
df_label.loc[(df_label.TP_Binary == True) & (df_label.TargetFilename.notna())]['TargetFilename'].drop_duplicates()

14806                                  C:\Users\pbeesly\ZZZZZZZZZZZZZZZZZZZZZ.ZZZ
34871                                 C:\Users\pbeesly\Downloads\coyn5igj.3io.pfx
45135                          C:\Users\pbeesly\AppData\Roaming\OfficeSupplies.7z
54496                                                     C:\Windows\Temp\Rar.exe
101500                               C:\Users\pbeesly\AppData\Roaming\working.zip
127838                                              C:\Windows\Temp\sdelete64.exe
152745                                                 C:\Windows\Temp\python.exe
163620                                 C:\Users\pbeesly\AppData\Roaming\Draft.Zip
166058                                      C:\Users\pbeesly\Downloads\monkey.png
173562                               C:\Users\pbeesly\ZZZZZZZZZZZZZZZZZZZZZZZ.ZZZ
174599                                       C:\Users\pbeesly\Desktop\working.zip
180348    C:\ProgramData\Microsoft\Windows\Start Menu\Programs\StartUp\hostui.lnk
189173          

In [227]:
df_label.loc[(df_label.TargetFilename.notna())]['EventID'].unique()

array([11., 23.,  2., 15.])

In [228]:
#df_label.loc[(df_label.TP_Binary == True)]['TargetFilename'].drop_duplicates()
df_label.loc[(df_label.TP_Binary == True)].groupby(['TargetFilename'])['EventID'].unique()

TargetFilename
C:\ProgramData\Microsoft\Windows\Start Menu\Programs\StartUp\hostui.lnk          [11.0]
C:\Users\pbeesZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ.ZZZ                                 [23.0]
C:\Users\pbeesly\AppData\Roaming\Draft.Zip                                       [11.0]
C:\Users\pbeesly\AppData\Roaming\OfficeSupplies.7z                               [11.0]
C:\Users\pbeesly\AppData\Roaming\working.zip                                     [11.0]
C:\Users\pbeesly\Desktop\working.zip                                             [23.0]
C:\Users\pbeesly\Downloads\coyn5igj.3io.pfx                                      [11.0]
C:\Users\pbeesly\Downloads\monkey.png                                            [11.0]
C:\Users\pbeesly\ZZZZZZZZZZZZZZZZZZZZZ.ZZZ                                       [23.0]
C:\Users\pbeesly\ZZZZZZZZZZZZZZZZZZZZZZZ.ZZZ                                     [23.0]
C:\Windows\Temp\Rar.exe                                                    [23.0, 11.0]
C:\Windows\Temp\p

In [241]:
# TargetFilename and Image could be solid pivot fields.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[df_label.Image == 'C:\Windows\Temp\sdelete64.exe'].groupby(['EventID']).count())

,@timestamp,@version,Category,Channel,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,E

In [243]:
bad_files = df_label[(df_label.TP_Binary==True) & (df_label.TargetFilename.notna())]['TargetFilename'].unique()

In [244]:
bad_files

array(['C:\\Users\\pbeesly\\ZZZZZZZZZZZZZZZZZZZZZ.ZZZ',
       'C:\\Users\\pbeesly\\Downloads\\coyn5igj.3io.pfx',
       'C:\\Users\\pbeesly\\AppData\\Roaming\\OfficeSupplies.7z',
       'C:\\Windows\\Temp\\Rar.exe',
       'C:\\Users\\pbeesly\\AppData\\Roaming\\working.zip',
       'C:\\Windows\\Temp\\sdelete64.exe',
       'C:\\Windows\\Temp\\python.exe',
       'C:\\Users\\pbeesly\\AppData\\Roaming\\Draft.Zip',
       'C:\\Users\\pbeesly\\Downloads\\monkey.png',
       'C:\\Users\\pbeesly\\ZZZZZZZZZZZZZZZZZZZZZZZ.ZZZ',
       'C:\\Users\\pbeesly\\Desktop\\working.zip',
       'C:\\ProgramData\\Microsoft\\Windows\\Start Menu\\Programs\\StartUp\\hostui.lnk',
       'C:\\Users\\pbeesZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ.ZZZ'], dtype=object)

In [245]:
df_label['THE_Binary'].value_counts()

False    174628
True      26272
Name: THE_Binary, dtype: int64

In [246]:
df_label['THE_Binary'].loc[(df_label.THE_Binary == False) & (df_label.TargetFilename.isin(bad_files))] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [253]:
# Looks like bad_processes covered all records with a bad_file in TargetFilename. Interesting.
df_label['THE_Binary'].value_counts()

False    174628
True      26272
Name: THE_Binary, dtype: int64

In [254]:
df_label['THE_Binary'].loc[(df_label.THE_Binary == False) & (df_label.Image.isin(bad_files))] = True

In [255]:
# Woof. Either ProcessGuid is a dominant selector OR I'm not implementing this label properly.
df_label['THE_Binary'].value_counts()

False    174628
True      26272
Name: THE_Binary, dtype: int64

In [ ]:
# What about bad registry keys?
# Woah Kernel Crash.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[(df_label.EventID == 12)].groupby(['EventID', 'Image', 'TargetObject']).count())